# EDA
Automatic EDA using ydata-profiling

# User params

In [1]:
file_name = "merged_dataset.parquet"
max_samples = 5_000

# Imports

In [2]:
import os
import pandas as pd
from ydata_profiling import ProfileReport
from src.utils.io.io import load_dataframe

In [3]:
base_folder = os.path.join('..', 'data', 'raw')
output_folder = os.path.join('..', 'data', 'output')

# create paths
file_path = os.path.join(base_folder, file_name)
profiling_path = os.path.join(output_folder, file_name.split('.')[0]+"_profiling.html")

# read json file into dataframe
df = load_dataframe(file_path)
print(f"Dataset shape: {df.shape}")

# display
display(df.head(1))


2024-08-07 14:14:33,998 - src.config - DEBUG - calling load_dataframe
2024-08-07 14:14:34,000 - src.config - INFO - Current working directory: c:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers\notebooks


,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_metadata,videos,store,categories,details,bought_together,subtitle,author
0,5.0,Work great,work great. use a new one every month,[],B01N0TQ0OH,B01N0TQ0OH,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1519317108692,0,True,...,[],9.99,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Mr Coffee Water Filter Instruction...,Geesta,"[Small Appliance Parts & Accessories, Coffee &...","{""Manufacturer"": ""Geesta"", ""Part Number"": ""Gee...",None,None,None


# Automated analysis : Report

In [4]:
# review report
df_to_analyze = df.sample(max_samples)
df_to_analyze['price'] = pd.to_numeric(df_to_analyze['price'], errors='coerce')
df_to_analyze['rating'] = pd.to_numeric(df_to_analyze['rating'], errors='coerce')

profile_review = ProfileReport(df_to_analyze, title="Profiling Report", minimal=False)
profile_review.to_file(profiling_path)

2024-08-07 14:14:54,686 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): packages.ydata.ai:443
2024-08-07 14:14:55,892 - urllib3.connectionpool - DEBUG - https://packages.ydata.ai:443 "GET /ydata-profiling?version=v4.9.0&python_version=3.12&report_type=regular&dataframe=spark&datatype=tabular&os=Windows&gpu=False HTTP/11" 200 None


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\alexi\AppData\Local\pypoetry\Cache\virtualenvs\reviewoutliers-SwZO3ms--py3.12\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
c:\Users\alexi\AppData\Local\pypoetry\Cache\virtualenvs\reviewoutliers-SwZO3ms--py3.12\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
c:\Users\alexi\AppData\Local\pypoetry\Cache\virtualenvs\reviewoutliers-SwZO3ms--py3.12\Lib\sit

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

2024-08-07 14:24:49,678 - matplotlib.colorbar - DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x000001C1D2CD9D90>


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Interactive Analysis

In [5]:
# import qgrid
# widget = qgrid.show_grid(df)
# widget

In [6]:
# from pivottablejs import pivot_ui
# pivot_ui(df)

# from pandasgui import show
# show(df)

# import tabloo
# tabloo.show(df)

# import dtale
# dtale.show(df)

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# from itables import show
# show(df, buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

# import pygwalker as pyg
# walker = pyg.walk(df)

# ipyaggrid

# Manual Analysis

In [7]:
from regex import D


def plot_counts(col_name: str) -> None:
    '''
    hist and stats on col that have arrays of str
    '''
    len_col = 'list_length_'+col_name
    
    print(f"Data type: {type(df[col_name].loc[0])}")
    
    # check the length of the list
    df[len_col] = df[col_name].apply(len)
    print("Counts : ", df[len_col].value_counts()[:4])

    import matplotlib.pyplot as plt
    plt.figure(figsize=(3, 2))
    plt.hist(df[len_col], bins=range(1, df[len_col].max() + 2), edgecolor='black')
    plt.xlabel('Length of List')
    plt.ylabel('Frequency')
    plt.title('Histogram of List Lengths')
    plt.show()

    # examples of len 1
    try:
        print("examples of len 1")
        display(df[df[len_col] == 1][col_name].sample(2))
    except:
        pass

    # examples of len superior to 1
    try:
        list_samples = df[df[len_col] > 1][col_name].sample(1).to_list()
        print("example of len > 1 : ", list_samples[0])
    except:
        pass
    
    

In [8]:
plot_counts('features')

Data type: <class 'numpy.ndarray'>
Counts :  list_length_features
5    402551
4     80227
3     66760
6     46694
Name: count, dtype: int64
examples of len 1


C:\Users\alexi\AppData\Local\Temp\ipykernel_2932\103120153.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


90999                [See details on "Product Description"]
156791    [IMV708 W10408179 4389177 Whirlpool Kitchenaid...
Name: features, dtype: object

example of len > 1 :  ['Your Best Choice ‘Washer Door Prop’ - the front load washing machine door prop helps keep your washing machine door open. It ensures that your washing machine is dry and ventilated so its gasket won’t get moldy, and can eliminate unpleasant smells. When you use this door stopper holder, your washing machine will stay clean. You no longer need to clean it often or buy expensive cleaners.'
 'Must Accessory for Front Load Washer - This front loader washer door prop is made of high-quality soft TPU Material and Powerful suction, Contains 3* suction Metal patch (with adhesive power), has no peculiar smell, will not rust, is compact, flexible and portable and has elasticity, and has zero damage to the washing machine.'
 'Strong Adhesion - Fits all Washing Machines,even if the washer door or door frame is non-magnetic.This product is equipped with 3 adhesive magnet pieces,helps you to easily handle any non magnet washing machine door.'
 'Easy to Install - 1. If the was

In [9]:
plot_counts('description')

Data type: <class 'numpy.ndarray'>
Counts :  list_length_description
0    328697
1    196277
4    166358
2     14959
Name: count, dtype: int64
examples of len 1


C:\Users\alexi\AppData\Local\Temp\ipykernel_2932\103120153.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


364935    [This is a genuine replacement part. The model...
333994    [HOLMES, JARDEN Cool Mist HWF62 Replacement Fi...
Name: description, dtype: object

example of len > 1 :  ['Product Description'
 'This is a Genuine Replacement Part, The Model Number and Name for The Following Item: Whirlpool (WHIRA) W10206287 Valve, Burner.'
 'From the Manufacturer'
 'This is a Genuine Replacement Part,The Model Number and Name for The Following Item:Whirlpool (WHIRA) W10206287 Valve, Burner']


In [10]:
display(df['images_metadata'].iloc[0])

{'hi_res': array(['https://m.media-amazon.com/images/I/71zqD75W03L._AC_SL1100_.jpg',
        'https://m.media-amazon.com/images/I/71qKIfx8dJS._AC_SL1500_.jpg',
        'https://m.media-amazon.com/images/I/81Yfvtag2FL._AC_SL1500_.jpg',
        'https://m.media-amazon.com/images/I/61dKJgtTOZL._AC_SL1100_.jpg',
        'https://m.media-amazon.com/images/I/71lSoy278RL._AC_SL1500_.jpg',
        'https://m.media-amazon.com/images/I/810wbaIjauL._AC_SL1500_.jpg',
        'https://m.media-amazon.com/images/I/61CwfTVj9dS._AC_SL1500_.jpg',
        'https://m.media-amazon.com/images/I/71AETO3-c9S._AC_SL1500_.jpg'],
       dtype=object),
 'large': array(['https://m.media-amazon.com/images/I/51qraA+1U-L._AC_.jpg',
        'https://m.media-amazon.com/images/I/41LAR3m2Z2S._AC_.jpg',
        'https://m.media-amazon.com/images/I/41y8sd6nqHL._AC_.jpg',
        'https://m.media-amazon.com/images/I/417x0xB6wXL._AC_.jpg',
        'https://m.media-amazon.com/images/I/41bnJQ1a3lL._AC_.jpg',
        'https://m

In [11]:
plot_counts('categories')

Data type: <class 'numpy.ndarray'>
Counts :  list_length_categories
4    576889
2     97441
3     95727
5      1784
Name: count, dtype: int64
examples of len 1


C:\Users\alexi\AppData\Local\Temp\ipykernel_2932\103120153.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


example of len > 1 :  ['Appliances' 'Parts & Accessories' 'Range Parts & Accessories' 'Burners']


In [12]:
plot_counts('details')

Data type: <class 'str'>
Counts :  list_length_details
2       8926
875     7487
1361    7020
553     6786
Name: count, dtype: int64


C:\Users\alexi\AppData\Local\Temp\ipykernel_2932\103120153.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


examples of len 1
example of len > 1 :  {"Material": "Carbon,Coconut Shell", "Product Dimensions": "6\"D x 6\"W x 10\"H", "Item Weight": "2.74 Pounds", "Duration": "6 \tmonths", "External Testing Certification": "IAPMO, NSF, WQA", "Brand": "PURELINE", "Manufacturer": "PURELINE", "Part Number": "mwf", "Item model number": "PL-100-S", "Is Discontinued By Manufacturer": "No", "Size": "3 Pack", "Color": "White", "Item Package Quantity": "1", "Special Features": "Perfect Compatibility and Easy Installation, Creates Clean and Great Tasting Water, NSF 372 Certified, NSF 42 Certified", "Batteries Included?": "No", "Batteries Required?": "No", "Warranty Description": "Life Time Warranty-100% Money Back Guaranteed. We offer 90 days return and 180 days replace (at no charge) after the date of original purchase for risk-free purchase.", "Best Sellers Rank": {"Tools & Home Improvement": 2485, "In-Refrigerator Water Filters": 75}, "Date First Available": "August 17, 2016"}


In [13]:
# plot_counts('bought_together')
display(df['bought_together'].iloc[0])

None

# Information from the documentation


- Pure IDs (0-Core): Complete dataset
- Pure IDs (5-Core): 
    - Each user must have rated at least 5 different items.
    - Each item must have been rated by at least 5 different users.

# Conclusion

## Overall


## Metadata

- `main_category`(str - categorical): could be considered categorical
- `title` (str): text
- `average_rating` (float): numerical
- `rating_number` (int): numerical
- `store` (str): names of shops could be considered categorical data
- `parent_asin`(str - ID): unique string identifier
- `features`(list of str): could be concatenated
- `categories`: (list of str): product category path. could be spread on several columns or concatenated
- `description` (list of str): mostly empty
- `details` (dict str - str): various product details
- `price` (float): subscription boxes had empty prices. rest are ok

To drop:
- `images` (list of str): too complex
- `videos` (list of str): too complex
- `bought_together` (boolean): mostly empty
- other that are category specific : `author`, `subtitle`, ...


Notes : 
- no duplicate rows

## Review

- `rating`: numerical
- `title`: text
- `text`: text
- `asin`: unique string identifier
- `parent_asin`: unique string identifier
- `user_id`: unique string identifier
- `timestamp`: numerical
- `helpful_vote`: numerical
- `verified_purchase`: boolean

To drop:
- `images`: list of image URLs. -> dropped

Notes : 
- there are duplicate rows / aka reviews. -> Drop duplicates
